# Sequence Hacking, Hashing, and Slicing


## Vector Take #1: Vector2d Compatible

In [30]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self) -> str:
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
         return (bytes([ord(self.typecode)]) +
                 bytes(self._components))
         
    def __eq__(self, __o: object) -> bool:
        return tuple(self) == tuple(__o)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def from_bytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]
    
    def __getattr__(self, name):
        cls = type(self)  #1 
        if len(name) == 1: #2
            pos = cls.shortcut_names.find(name) #3
        if 0 <= pos < len(self._components): #4
            return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' #5
        raise AttributeError(msg.format(cls, name))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1: # 1
            if name in cls.shortcut_names: # 2
                error = 'readonly attribute {attr_name!r}'
            elif name.islower(): # 3
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = '' # 4
            if error: # 5
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

1. The self._components instance “protected” attribute will hold an array with the Vector components
2. To allow iteration, we return an iterator over self._components
3. Use reprlib.repr() to get a limited-length representation of self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
4. Remove the array('d', prefix and the trailing ) before plugging the string into a Vector constructor call.
5. Build a bytes object directly from self._components
6. We can’t use hypot anymore, so we sum the squares of the components and compute the sqrt of that.
7. The only change needed from the earlier frombytes is in the last line: we passthe memoryview directly to the constructor, without unpacking with * as we did before.

## Protocols and Duck Typing

In the context of object-oriented programming, a protocol is an informal interface, defined only in documentation and not in code. For example, the sequence protocol in Python entails just the `__len__` and `__getitem__` methods. Any class Spam that implements those methods with the standard signature and semantics can be used anywhere a sequence is expected. Whether Spam is a subclass of this or that is irrelevant; all that matters is that it provides the necessary methods.


## Vector Take #2: A Sliceable Sequence

```python
class Vector:
    # many lines omitted
    # ...
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        return self._components[index]
```

In [3]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [4]:
v1[0], v1[-1]

(3.0, 5.0)

In [5]:
v7 = Vector(range(7))

In [6]:
v7[1:4]

array('d', [1.0, 2.0, 3.0])

It would be better if a slice of a Vector was also a Vector instance and not a array. To make Vector produce slices as Vector instances, we can’t just delegate the slicing to array. We need to analyze the arguments we get in `__getitem__` and do the right thing.

### How Slicing Works


In [7]:
class MySeq:
    def __getitem__(self, index):
        return index

In [8]:
s = MySeq()
s[1]

1

In [9]:
s[1:4]

slice(1, 4, None)

In [10]:
s[1:4:2]

slice(1, 4, 2)

In [11]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [12]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

1. For this demonstration, `__getitem__` merely returns whatever is passed to it
2. slice(1, 4, 2) means start at 1, stop at 4, step by 2.
3. Surprise: the presence of commas inside the [] means `__getitem__` receives a tuple. The tuple may even hold several slice objects.

Inspecting a slice we find the data attributes start, stop, and step, and an indices method.

In [14]:
print(dir(slice))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


### A Slice-Aware `__getitem__`

```py
def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    cls = type(self) # 1
    if isinstance(index, slice): # 2
         return cls(self._components[index]) # 3
    elif isinstance(index, numbers.Integral): # 4
        return self._components[index] # 5
    else:
        msg = '{cls.__name__} indices must be integers'
        raise TypeError(msg.format(cls=cls)) # 6
```

1. Get the class of the instance for later use
2. If the index argument is a slice...
3. invoke the class to build nother Vector instance from a slice of the _components array
4. If the index is an int or some other kind of integer...
5. just return the specific item from _components
6. otherwise raise an exception

## Vector Take #3: Dynamic Attribute Access

It may be convenient to access the first few components with shortcut letters such as x, y, z instead of v[0], v[1] and v[2]

```py

shortcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self)  #1 
    if len(name) == 1: #2
        pos = cls.shortcut_names.find(name) #3
    if 0 <= pos < len(self._components): #4
        return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}' #5
    raise AttributeError(msg.format(cls, name))
```

1. Get vector class for later use
2. If the name is one character, it maye be one of the shortcut_names
3. find position of 1-letter name
4. if the position is within range, return the array element
5. if either test failed, raise AttributeError

In [24]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [26]:
v.x

0.0

In [27]:
v.x = 10
v.x

10

In [28]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

1. Access element v[0] as v.x
2. Assign new value to v.x. This should raise an exception
3. reading v.x shows the new value, 10
4. However, the vector components did not change

Python only calls `__getattr__` as a fall back, when the object does not have the named attribute. However, after we assign v.x = 10, the v object now has an x attribute, so `__getattr__` will no longer be called to retrieve v.x: the interpreter will just return the value 10 that is bound to v.x. On the other hand, our implementation of `__getattr__` pays no attention to instance attributes other than self._components, from
where it retrieves the values of the “virtual attributes” listed in shortcut_names.

We need to customize the logic for setting attributes in our Vector class in order to avoid this inconsistency.



```py
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1: # 1
        if name in cls.shortcut_names: # 2
            error = 'readonly attribute {attr_name!r}'
        elif name.islower(): # 3
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = '' # 4
        if error: # 5
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
     super().__setattr__(name, value) # 6
```

1. Special handling for single-character attribute names
2. If name is one of the xyzt, set specific error message
3. if name is lowercase, set error message about all singe-letter names
4. otherwise set blank error message 
5. if there is a nonblank error  message raise AttributeError
6. Defalt case: call `__setattr__` on superclass for standard behavior



The super() function provides a way to access methods of super‐
classes dynamically, a necessity in a dynamic language support‐
ing multiple inheritance like Python. It’s used to delegate some task
from a method in a subclass to a suitable method in a superclass

very often when you implement `__getattr__` you need to code
`__setattr__` as well, to avoid inconsistent behavior in your objects.

If we wanted to allow changing components, we could implement `__setitem__` to enable v[0] = 1.1 and/or `__setattr__` to make v.x = 1.1 work.

## Vector Take #4: Hashing and a Faster ==


```py
from array import array
import reprlib
import math
import functools # 
import operator # 


class Vector:
    typecode = 'd'

    
    def __eq__(self, other): # 
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 
        return functools.reduce(operator.xor, hashes, 0) 
```


1. Import functools to use reduce
2. Import operator to use xor
3. Create a generator expression to lazily compute the hash of each component.
4. Feed hashes to reduce with the xor function to compute the aggregate hash value; the third argument, 0, is the initializer 


Another way to implement hash function with map, making the mapping step more visible.

```py
def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)

```

Rethinking the  `__eq__` implemented as:
```py 
def __eq__(self, other):
     return tuple(self) == tuple(other)
```
Very inefficient for Vector  instances that may have thousands off components, as it will have to build two tuples copying all the values to compare.

```py
def __eq__(self, other):
    if len(self) != len(other): # 
        return False
    for a, b in zip(self, other): # 
        if a != b: # 
            return False
    return True 

```

1. If len of the objects are different, they ar not equal
2. zip produces a generator of tuples made from the items in each iterable argument.
3. As soon as two components are different, exit return False
4. Otherwise, the objects are equal

Even simpler with all()

```py
def __eq__(self, other):
     return len(self) == len(other) and all(a == b for a, b in zip(self, other))
```

# Chapter Summary

1. The fact that Vector behaves as a sequence just by implementing `__getitem__` and `__len__` prompted a discussion of protocols, the informal interfaces used in duck-typed languages4
2. returning new Vector instances, just like a Pythonic sequence is expected to do.
3. provide read-only access to the first few Vector components by implementing `__getattr__`. implementing `__setattr__` as well, to forbid assigning values to single-letter attributes Very often, when you code a `__getattr__` you need to add `__setattr__` too, in order to avoid inconsistent behavior
4. Implementing the `__hash__` function provided the perfect context for using `functools.reduce`,  used the `all` reducing built-in to create a more efficient `__eq__` method.



